In [41]:
import numpy as np
import pyBeamSim
import pandas as pd

In [49]:
ref_energy = 5
num_particle = 102400
simulator = pyBeamSim.BeamSimulator()
# simulator.free_beam()
simulator.init_beam(num_particle, 938.272046, 1.0, 0.0)
simulator.set_beamTwiss(0, 0.003, 0.0001, 0, 0.003, 0.0001, 0, 8, 3.1415926e-11, 0, ref_energy, 500, 1)
simulator.save_initial_beam()

simulator.init_spacecharge()

"""" parameters random """
param_list = []
result_list = []
result_sig4 = []

In [50]:
param_name = ['quad1_length', 'quad1_gradient', 'quad2_length', 'quad2_gradient', 'quad3_length',
             'quad3_gradient', 'quad4_length', 'quad4_gradient', 'quad5_length', 'quad5_gradient']
for i in range(100):

    simulator.restore_initial_beam()
    # key
    simulator.UpdateBeamParameters()

    param = {}
    for j in range(5):
        param[f'quad{j+1}_length'] = np.random.uniform(0.1, 0.5)
        param[f'quad{j+1}_gradient'] = np.random.uniform(-20, 20)

    param_list.append(dict(zip(param_name, list(param.values()))))

    """beamline initial """

    simulator.init_Beamline()  #needed without interface

    simulator.add_Drift('drift1', 0.19, 0.05)
    simulator.add_Quad('quad1', param['quad1_length'], 0.012, param['quad1_gradient'])
    simulator.add_Drift('drift2', 0.058, 0.05)
    simulator.add_Quad('quad2', param['quad2_length'], 0.028, param['quad2_gradient'])
    simulator.add_Drift('drift3', 0.058, 0.05)
    simulator.add_Quad('quad3', param['quad3_length'], 0.028, param['quad3_gradient'])
    simulator.add_Drift('drift4', 3.365, 0.05)
    # simulator.add_Bend('bend1', 0.511, 0.03, 0.7854, 0, 0,
    #                    0, 1.0, 5)
    simulator.add_Drift('drift5', 1.365, 0.05)
    # simulator.add_ApertureRectangular('aper1', 0.01, 0.01, 0.1, 0.1)
    simulator.add_Drift('drift6', 0.2, 0.05)
    simulator.add_Quad('quad4', param['quad4_length'], 0.028, param['quad4_gradient'])
    simulator.add_Drift('drift7', 0.2, 0.05)
    simulator.add_Quad('quad5', param['quad5_length'], 0.028, param['quad5_gradient'])
    simulator.add_Drift('drift8', 1.015, 0.05)


    # generate the dataframe
    envelope = simulator.simulate_and_getEnvelope(use_spacecharge=True)

    x_avg = envelope['Avg_x']
    x_sig = envelope['Sig_x']
    y_sig = envelope['Sig_y']

    # need a function to get twiss params
    x_avg0, x_avg1, x_avg2, x_avg3, x_avg4 = x_avg[0], x_avg[2], x_avg[4], x_avg[10], x_avg[13]

    x_avg_results = {
        'x_avg0': x_avg0,
        'x_avg1': x_avg1,
        'x_avg2': x_avg2,
        'x_avg3': x_avg3,
        'x_avg4': x_avg4
    }

    sig_results = {
        'x_sig4': x_sig[4],
        'y_sig4': y_sig[4]
    }

    result_list.append(x_avg_results)
    result_sig4.append(sig_results)

In [44]:
result_list

[{'x_avg0': -2.582749882028499e-09,
  'x_avg1': 8.61498528826921e-07,
  'x_avg2': 4.099216247350977e-05,
  'x_avg3': -0.00012508344539323248,
  'x_avg4': 0.0014655153342083365},
 {'x_avg0': -2.582749882028499e-09,
  'x_avg1': 1.6711284795167465e-07,
  'x_avg2': 1.8313107524399366e-06,
  'x_avg3': -0.000193412722694963,
  'x_avg4': 0.001130747369563913},
 {'x_avg0': -2.582749882028499e-09,
  'x_avg1': -1.8324628205302348e-07,
  'x_avg2': 5.363428373808256e-05,
  'x_avg3': 0.0,
  'x_avg4': 0.0},
 {'x_avg0': -2.582749882028499e-09,
  'x_avg1': 1.4352233493269362e-06,
  'x_avg2': 1.8143377260774787e-05,
  'x_avg3': 9.738231344043764e-05,
  'x_avg4': 0.0},
 {'x_avg0': -2.582749882028499e-09,
  'x_avg1': -4.76060204190494e-06,
  'x_avg2': -9.935923186680442e-05,
  'x_avg3': 0.0007682790014346675,
  'x_avg4': 0.0},
 {'x_avg0': -2.582749882028499e-09,
  'x_avg1': 2.5790702891762385e-06,
  'x_avg2': 6.733383804065043e-05,
  'x_avg3': -0.00011641303103108176,
  'x_avg4': -0.0021481921885548793},

In [51]:
df_param = pd.DataFrame(param_list)
df_result = pd.DataFrame(result_list)
df = pd.concat((df_param, df_result), axis=1)

In [53]:
df_result

,x_avg0,x_avg1,x_avg2,x_avg3,x_avg4
0,-2.582750e-09,1.377982e-07,0.000105,0.000000,0.000000
1,-2.582750e-09,4.985111e-06,0.000119,0.000043,0.000000
2,-2.582750e-09,1.783615e-07,-0.000076,0.000000,0.000000
3,-2.582750e-09,-6.132881e-07,-0.000018,0.000122,0.000000
4,-2.582750e-09,-5.189361e-06,-0.000130,0.000000,0.000000
...,...,...,...,...,...
95,-2.582750e-09,1.202924e-06,-0.000006,-0.000825,0.000000
96,-2.582750e-09,3.629682e-06,-0.000052,-0.000320,0.000000
97,-2.582750e-09,-3.107383e-06,-0.000154,-0.000437,0.000000
98,-2.582750e-09,-5.057724e-07,-0.000063,-0.003010,-0.000044
